## Using Neural Nets to recognize Handwritten Digits 

In [17]:
import numpy as np
import random

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [18]:
net = Network([2, 3, 1])

In [19]:
net.SGD(training_data, epochs=30, mini_batch_size=10, eta=3.0, test_data=test_data)

ValueError: shapes (3,2) and (784,1) not aligned: 2 (dim 1) != 784 (dim 0)

In [20]:
import numpy as np
import pickle
import gzip
import os
from urllib import request

def load_mnist():
    """Download and load the MNIST dataset"""
    # Download MNIST if not present
    url = "https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz"
    mnist_path = "mnist.pkl.gz"
    
    if not os.path.exists(mnist_path):
        print("Downloading MNIST dataset...")
        request.urlretrieve(url, mnist_path)
    
    # Load the dataset
    with gzip.open(mnist_path, 'rb') as f:
        try:
            training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
        except:
            training_data, validation_data, test_data = pickle.load(f)
    
    return format_data(training_data), format_data(validation_data), format_data(test_data)

def format_data(data):
    """Format data for the neural network"""
    inputs = [np.reshape(x, (784, 1)) for x in data[0]]
    # Convert the outputs to one-hot vectors
    outputs = [vectorized_result(y) for y in data[1]]
    return list(zip(inputs, outputs))

def vectorized_result(j):
    """Convert a digit to a 10-dimensional one-hot vector"""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# Example usage
if __name__ == "__main__":
    # Load and format MNIST data
    print("Loading MNIST data...")
    training_data, validation_data, test_data = load_mnist()
    
    # Create the network with 784 input neurons (28x28 pixels),
    # 30 hidden neurons, and 10 output neurons (one for each digit)
    net = Network([784, 30, 10])
    
    # Train the network
    print("Starting training...")
    net.SGD(training_data, 
            epochs=30, 
            mini_batch_size=10, 
            eta=3.0,
            test_data=test_data)

Loading MNIST data...
Starting training...
Epoch 0: 9041 / 10000
Epoch 1: 9160 / 10000
Epoch 2: 9272 / 10000
Epoch 3: 9258 / 10000
Epoch 4: 9313 / 10000
Epoch 5: 9412 / 10000
Epoch 6: 9418 / 10000
Epoch 7: 9402 / 10000
Epoch 8: 9424 / 10000
Epoch 9: 9433 / 10000
Epoch 10: 9382 / 10000
Epoch 11: 9451 / 10000
Epoch 12: 9446 / 10000
Epoch 13: 9455 / 10000
Epoch 14: 9466 / 10000
Epoch 15: 9484 / 10000
Epoch 16: 9485 / 10000
Epoch 17: 9480 / 10000
Epoch 18: 9486 / 10000
Epoch 19: 9486 / 10000
Epoch 20: 9472 / 10000
Epoch 21: 9491 / 10000
Epoch 22: 9487 / 10000
Epoch 23: 9486 / 10000
Epoch 24: 9505 / 10000
Epoch 25: 9482 / 10000
Epoch 26: 9515 / 10000
Epoch 27: 9495 / 10000
Epoch 28: 9501 / 10000
Epoch 29: 9501 / 10000
